**INTRODUÇÃO**

Este notebook traz análise reallizada para a reportagem [Cancioneiro das múltiplas Fortalezas](https://mais.opovo.com.br/reportagens-especiais/2022/04/12/as-multiplas-fortalezas-expressas-em-cancoes.html), publicada no O POVO+ no dia 12 de abril de 2022.

A reportagem foi produzida em alusão ao aniversário de 296 anos de Fortaleza a partir de dados utilizados no [aplicativo Fortaleza em Música](https://play.google.com/store/apps/details?id=com.ufc.fortalezaemmusica&gl=US).

O projeto é desenvolvido pelo Grupo de Imagem, Consumo e Experiência Urbana (GICEU), vinculado ao Programa de Pós-Graduação em Comunicação do Instituto de Cultura e Arte da Universidade Federal do Ceará (ICA/UFC).

A partir dos arquivos concedidos pelo grupo de pesquisa da UFC, a Central de Jornalismo de Dados do **O POVO** (DATADOC) estruturou a base de dados e realizou a análise a seguir.

Os dados originais pertencem ao Grupo de Imagem, Consumo e Experiência Urbana (ICA/UFC) e por isso não será disponibilizada neste repositório.

A análise é feita em Python e este notebook traz os seguintes tópicos:

1. [Importação e tratamento](#Importação)
2. [Criação de nuvens de palavras](#NLTK)

  2.1 [Criação de nuvem de palavras geral](#geral)

  2.2 [Criação de nuvem de palavras das inspirações dos autores](#inspiração)
3. [Encontrando os bairros de Fortaleza mais cantados ](#bairros)

  3.1 [Quantidade de músicas por bairros](#quantidade)

  3.2 [Músicas por bairro e por período](#período)



**EXPEDIENTE**

**Edição DATADOC**

Thays Lavor


**Análise e visualização de dados**

Gabriela Custódio e Alexandre Cajazeira

In [ ]:
!pip install geopandas
!pip install nltk

     |████████████████████████████████| 1.0 MB 14.7 MB/s 
     |████████████████████████████████| 6.3 MB 55.4 MB/s 
     |████████████████████████████████| 16.7 MB 67.5 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPolygon
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import plotly.express as px

from googleapiclient.discovery import build

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Importação e tratamento <a name="Importação"></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
arquivo = pd.read_csv('caminho_para_o_dataset')

## Uso da biblioteca NLTK para criação de nuvens de palavras <a name="NLTK"></a>

###  criação de nuvem de palavras geral  <a name="geral"></a>

In [ ]:
# excluir linhas com letras de músicas faltantes

arquivo_letras = arquivo[arquivo['Letra'].isna() == False]

In [ ]:
# lista de stopwords em pt-br

stopwords_ptbr = stopwords.words('portuguese')

# ampliar a lista de stopwords com outras expressões que estão presentes nas letras das músicas
novas_palavras = ['vai', 'então', '2x', 'ti', 'ia', 'ai', 'la', 'iaiou', 'lará', 'pra', 'dê', 'tão', 'ja', '3x', '5x', '315', 'abc', 'tô', 'cês', 'fic', 'tá', 'pro', 'deu', 'ê', 'mim', '7x', 'y', 'ô', '4x', 'n', '116', 'br', '20', '10%', 'né', 'aonde', 'en', 'refrão', 'pa', 'las', 'nê', 'iaiou', 'qua', '26', 'os', '10', '07', '7', '051', 'lá', '12', '8x', 'aí', '1604', 'ó', 'ah', 'porque', 'sobre', 'assim', '1941', '2020', '2021','1980', '1902', '1987', '2015', '2018', '1991', '2013', '1994', '1978', 'uh', 'porque', 'pra', 'pras', 'pro', 'pros', 'têm', 'tô', 'falando', 'assim', 'né', 'através', 'ela', 'ele', 'sobre', 'tanto', 'tal', 'pô', 'n', 'dez',  'por', 'em', 'ali', 'né', 'é', 'talvez', 'então', 'assim', 'fala', 'digo', 'poxa', 'aí', 'entendeu', 'entende', 'a', 'x', 'de', 'tava', 'onde', 'acho']

stopwords_ptbr.extend(novas_palavras)

In [ ]:
# criar função processingTxt para preservar termos compostos

def processingTxt(txt):
    return txt.replace('beira mar', 'beira_mar').replace('beira-mar', 'beira_mar').replace('praia de iracema', 'praia_de_iracema').replace('castelo encantado', 'castelo_encantado').replace('cidade 2000', 'cidade_2000').replace('barra do ceará', 'barra_do_ceará').replace('rio ceará', 'rio_ceará').replace('vitória-régia', 'vitória_régia').replace('são paulo', 'são_paulo').replace('praia do futuro', 'praia_do_futuro').replace('gá-gá', 'gá_gá').replace('ray-ban', 'ray_ban').replace('high tech', 'high_tech').replace('bloco bate palmas', 'bloco_bate_palmas').replace('a gente', 'a_gente').replace('água doce', 'água_doce').replace('estrela do mar', 'estrela_do_mar').replace('caminho das índias', 'caminho_das_índias').replace('matias beck', 'matias_beck').replace('moinhos de holanda', 'moinhos_de_holanda').replace('praia do peixe', 'praia_do_peixe').replace('mina de prata', 'mina_de_prata').replace('alto mar', 'alto_mar').replace('pero coelho de sousa', 'pero_coelho_de_sousa').replace('martins soares moreno', 'martins_soares_moreno').replace('vila nova lisboa', 'vila_nova_lisboa').replace('morro de santiago', 'morro_de_santiago').replace('pôr-do-sol', 'pôr_do_sol').replace('berço da colonização', 'berço_da_colonização').replace('soares moreno', 'soares_moreno').replace('querer bem', 'querer_bem').replace('música baiana', 'música_baiana').replace('roqueiro do montese', 'roqueiro_do_montese').replace('beco sem saída', 'beco_sem_saída').replace('rock n roll', 'rock_n_roll').replace('zé bezerra', 'zé_bezerra').replace('ray-ban', 'ray_ban').replace('barruada gá-gá', 'barruada_gá_gá').replace('meu bem', 'meu_bem').replace('beira da praia', 'beira_da_praia').replace('caça e pesca', 'caça_e_pesca').replace('rio pajeú', 'rio_pajeú').replace('estrela do norte', 'estrela_do_norte').replace('cruzeiro do sul', 'cruzeiro_do_sul').replace('vila morena', 'vila_morena').replace('água boa', 'água_boa').replace('parque araxá', 'parque_araxá').replace('campo do pio', 'campo_do_pio').replace('são gerardo', 'são_gerardo').replace('dionísio torres', 'dionísio_torres').replace('instância castelo', 'instância_castelo').replace('monte castelo', 'monte_castelo').replace('aço de joão lopes', 'aço_de_joão_lopes').replace('antônio bezerra', 'antônio_bezerra').replace('barro vermelho', 'barro_vermelho').replace('amadeu furtado', 'amadeu_furtado').replace('franco rabelo', 'franco_rabelo').replace('curral das éguas', 'curral_das_éguas').replace('oitão preto', 'oitão_preto').replace('leste-oeste', 'leste_oeste').replace('rodolfo teófilo', 'rodolfo_teófilo').replace('nossa senhora das graças', 'nossa_senhora_das_graças').replace('henrique jorge', 'henrique_jorge').replace('casa popular', 'casa_popular').replace('álvaro weyne', 'álvaro_weyne').replace('santo antônio da floresta', 'santo_antônio_da_floresta').replace('fazenda boa vista', 'fazenda_boa_vista').replace('blocos de carnaval', 'blocos_de_carnaval').replace('esquina do samba', 'esquina_do_samba').replace('astro rei', 'astro_rei').replace('fora da lei', 'fora_da_lei').replace('praia da barra', 'praia_da_barra').replace('zé walter', 'zé_walter').replace('beira-mar', 'beira_mar').replace('nação miami', 'nação_miami').replace('arranha-céus', 'arranha_céus').replace('por que', 'por_que').replace('são cristóvão', 'são_cristóvão').replace('joão paulo', 'joão_paulo').replace('boa vista', 'boa_vista').replace('cachorro grande', 'cachorro_grande').replace('sítio timbaúba', 'sítio_timbaúba').replace('meia-noite', 'meia_noite').replace('meia noite', 'meia_noite').replace('praça do carmo', 'praça_do_carmo').replace('major facundo', 'major_facundo').replace('pedro pereira', 'pedro_pereira').replace('bloco bate palmas', 'bloco_bate_palmas').replace('bate-palmas', 'bate_palmas').replace('teatro flores do lixo', 'teatro_flores_do_lixo').replace('estrela do sertão', 'estrela_do_sertão').replace('dança de rua', 'dança_de_rua').replace('hip-hop', 'hip_hop').replace('conjunto palmeiras das artes', 'conjunto_palmeiras_das_artes').replace('solidária economia', 'solidária_economia').replace('vitória-régia', 'vitória_régia').replace('unidos da cachorra', 'unidos_da_cachorra').replace('a doida é tu', 'a_doida_é_tu').replace('luz da vila', 'luz_da_vila').replace('som da lira', 'som_da_lira').replace('bons amigos', 'bons_amigos').replace('a turma do mamão', 'a_turma_do_mamão').replace('as gata pira', 'as_gata_pira').replace('agora vai', 'agora_vai').replace('amante de iracema', 'amante_de_iracema').replace('concentra mas não sai', 'concentra_mas_não_sai').replace("debaixo d'água", 'debaixo_dágua').replace('tupi or not tupi', 'tupi_or_not_tupi').replace('américa do sul', 'américa_do_sul').replace('quatro graus', 'quatro_graus').replace('br 116', 'br_116').replace('canção de bulbrax', 'canção_de_bulbrax').replace('ponte velha', 'ponte_velha').replace('jangada de vela', 'jangada_de_vela').replace('são paulo', 'são_paulo').replace('fim de semana', 'fim_de_semana').replace('sexta-feira', 'sexta_feira').replace('kiss me', 'kiss_me').replace('kill me', 'kill_me').replace('femme bateau', 'femme_bateau').replace('luiz assumpção', 'luiz_assumpção').replace('otávio bonfim', 'otávio_bonfim').replace('jardim américa', 'jardim_américa').replace('nova metrópole', 'nova_metrópole').replace('vinte três', 'vinte_três').replace('vinte e quatro', 'vinte_e_quatro').replace('décima primeira', 'décima_primeira').replace('vinte e um', 'vinte_e_um').replace('ildefonso albano', 'ildefonso_albano').replace('vigésima segunda', 'vigésima_segunda').replace('pio 12', 'pio_12').replace('conjunto ceará', 'conjunto_ceará').replace('dezenove e doze', 'dezenove_e_doze').replace('nova assunção', 'nova_assunção').replace('décima sexta', 'décima_sexta').replace('joão vinte e três', 'joão_vinte_e_três').replace('domila quinze', 'domila_quinze').replace('antônio bezerra', 'antônio_bezerra').replace('edson queiroz', 'edson_queiroz').replace('vinte e seis', 'vinte_e_seis').replace('costa barros', 'costa_barros').replace('henrique jorge', 'henrique_jorge').replace('vinte e sete', 'vinte_e_sete').replace('trinta e um', 'trinta_e_um').replace('vinte e nove', 'vinte_e_nove').replace('novo maracanaú', 'novo_maracanaú').replace('vinte e oito', 'vinte_e_oito').replace('trinta e três', 'trinta_e_três').replace('trigésima quarta', 'trigésima_quarta').replace('almirante tamandaré', 'almirante_tamandaré').replace('muito ruim', 'muito_ruim').replace('trinta e dois', 'trinta_e_dois').replace('bom jardim', 'bom_jardim').replace('fortalezas e suas dps', 'fortalezas_e_suas_dps').replace('blá-blá-blá', 'blá_blá_blá').replace('escrava sexual', 'escrava_sexual').replace('quatro ventos', 'quatro_ventos').replace('camisa de vênus', 'camisa_de_vênus').replace('mãe solteira', 'mãe_solteira').replace('meu deus', 'meu_deus').replace('fortaleza bela', 'fortaleza_bela').replace('alto mar', 'alto_mar').replace('estátua de iracema', 'estátua_de_iracema').replace('mucambo cafundó', 'mucambo_cafundó').replace('velas do mucuripe', 'velas_do_mucuripe').replace('planalto central', 'planalto_central').replace('nova iorque', 'nova_iorque').replace('beca xl', 'beca_xl').replace('zero sete', 'zero_sete').replace('beck xl', 'beck_xl').replace('double xl', 'double_xl').replace('high tech', 'high_tech').replace('bad vibes', 'bad_vibes').replace('a sete chaves', 'a_sete_chaves').replace('guerra às drogas', 'guerra_às_drogas').replace('seu mané', 'seu_mané').replace('rio de janeiro', 'rio_de_janeiro').replace('jangada são pedro', 'jangada_são_pedro').replace('getúlio vargas', 'getúlio_vargas').replace('dona maria', 'dona_maria').replace('grande otelo', 'grande_otelo').replace('castelo encantado', 'castelo_encantado').replace('zona leste', 'zona_leste').replace('cinema noir', 'cinema_noir').replace('menino de rua', 'menino_de_rua').replace('cidade 2000', 'cidade_2000').replace('praia do futuro', 'praia_do_futuro').replace('ao vivo', 'ao_vivo').replace('pedro bala', 'pedro_bala').replace('faixa de gaza', 'faixa_de_gaza').replace('gás lacrimogêneo', 'gás_lacrimogêneo').replace('bala de borracha', 'bala_de_borracha').replace('roda gigante', 'roda_gigante').replace('holocausto capital', 'holocausto_capital').replace('merci bocu', 'merci_bocu').replace('rua formosa', 'rua_formosa').replace('artigo 26', 'artigo_26').replace('galinha pedrês', 'galinha_pedrês').replace('south america', 'south_america').replace('nata do lixo', 'nata_do_lixo').replace('luxo da aldeia', 'luxo_da_aldeia')

In [ ]:
# deixar letras de músicas em caixa baixa, criar a coluna '['letras_musicas'] e aplicar função processingTxt

arquivo_letras['letras_musicas'] = arquivo_letras['Letra'].str.lower()
arquivo_letras['letras_musicas'] = arquivo_letras['letras_musicas'].fillna('-').apply(lambda x: processingTxt(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# criar função para gerar dataframe com palavras que mais se repetem e criar nuvem de palavras

def encontrarPalavras(letras):
    
    # criar lista com letras das músicas após retirar o separador
    letras_separadas = list()

    for letra_musica in letras:
        letra_musica = letra_musica.lower()
        letra_musica_separada = letra_musica.split(' / ')
        letras_separadas.append(letra_musica_separada)

    # criar variável para tokenizador
    tokenizador = nltk.WhitespaceTokenizer()

    # criar lista com letras das músicas após tokenização
    frases_letras_tokenizadas = list()

    for letra in letras_separadas:
        for frase in letra:
            frase_tokenizada = tokenizador.tokenize(frase)
            frases_letras_tokenizadas.append(frase_tokenizada)

    # criar lista com palavras sem pontuação e caracteres especiais
    palavras_sem_pontuacao = list()

    for frase in frases_letras_tokenizadas:
        for palavra in frase:
            nova_palavra = palavra.translate ({ord(c): "" for c in "…'‘“”!@#$%^&*()[]{};:.,/–-<>?\|`~=+"})
            palavras_sem_pontuacao.append(nova_palavra)

    # criar lista com palavras fora da lista de stopwords
    palavras_fora_stopwords = list()

    for palavra in palavras_sem_pontuacao:
        if palavra not in stopwords_ptbr:
            palavras_fora_stopwords.append(palavra)

    # criar dataframe com as palavras que não estão na lista de stopwords
    df_palavras_coluna = pd.DataFrame(palavras_fora_stopwords)

    df_palavras_coluna.rename(columns={0: 'palavras'}, inplace=True)
    df_palavras_coluna['repetições'] = 1
    
    # remover underline
    df_palavras_coluna['palavras'] = df_palavras_coluna['palavras'].str.replace('_', ' ')
    
    return df_palavras_coluna

In [ ]:
# aplicar função definida anteriormente na coluna arquivo_letras['letras_musicas']

nuvem_geral = encontrarPalavras(arquivo_letras['letras_musicas'])

In [ ]:
# agrupamento das palavras mais recorrentes e salvar em .csv para montar nuvem de palavras

df_nuvem_geral = nuvem_geral.groupby(by='palavras').sum().sort_values(by='repetições', ascending=False).reset_index()

In [ ]:
# salvar arquivo em .csv

df_nuvem_geral.to_csv('df_nuvem_palavras_geral.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
df_nuvem_geral.head()

,palavras,repetições
0,mar,60
1,sol,57
2,tempo,42
3,dia,41
4,vida,37


In [ ]:
# total de palavras
df_nuvem_geral['palavras'].nunique()

2811

In [ ]:
# total de palavras que se repetem
df_nuvem_geral[df_nuvem_geral['repetições'] != 1]['palavras'].nunique()

1196

In [ ]:
# total de palavras com ocorrência única
df_nuvem_geral[df_nuvem_geral['repetições'] == 1]['palavras'].nunique()

1615

###  criação de nuvem de palavras das inspirações <a name="inspiração"></a>


In [ ]:
# delimitar coluna com nome da música e inspiração
df_inspiracao = arquivo_letras[['Nome da música', 'inspiração_copia']]

In [ ]:
pd.set_option('display.max_rows', 70)

In [ ]:
df_inspiracao.dropna(inplace=True)
df_inspiracao.drop_duplicates(subset='inspiração_copia', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
df_inspiracao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 78
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Nome da música    73 non-null     object
 1   inspiração_copia  73 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [ ]:
# criar função processingTxtInspiracao para preservar termos compostos na coluna de depoimentos sobre a inspiração das músicas

def processingTxtInspiracao(txt):
    return txt.replace('1000 reais', '1000_reais').replace('beira mar', 'beira_mar').replace('beira-mar', 'beira_mar').replace('praia de iracema', 'praia_de_iracema').replace('rio ceará', 'rio_ceará').replace('vitória-régia', 'vitória_régia').replace('arranha-céu', 'arranha_céu').replace('são paulo', 'são_paulo').replace('praia do futuro', 'praia_do_futuro').replace('a gente', 'a_gente').replace('13 de maio', '13_de_maio').replace('19 anos', '19_anos').replace('61 dias', '61_dias').replace('abertura política', 'abertura_política').replace('anos 1980', 'anos_1980').replace('anos 2010', 'anos_2010').replace('anos 70', 'anos_70').replace('augusto pontes', 'augusto_pontes').replace('avenida santos dumont', 'avenida_santos_dumont').replace('balneabilidade livre', 'balneabilidade_livre').replace('banco palmas', 'banco_palmas').replace('bar academia', 'bar_academia').replace('bar do anísio', 'bar_do_anísio').replace('bares do benfica', 'bares_do_benfica').replace('barra do ceará', 'barra_do_ceará').replace('barra nova', 'barra_nova').replace('barra pesada', 'barra_pesada').replace('base aérea de fortaleza', 'base_aérea_de_fortaleza').replace('belas artes', 'belas_artes').replace('benfica na boca da noite', 'benfica_na_boca_da_noite').replace('bertolt brecht', 'bertolt_brecht').replace('bloco bate palmas', 'bloco_bate_palmas').replace('bloco prazer', 'bloco_prazer').replace('br-116', 'br_116').replace('brasil de dentro', 'brasil_de_dentro').replace('brega sabbath', 'brega_sabbath').replace('briga de gangue', 'briga_de_gangue').replace('bárbara de alencar', 'bárbara_de_alencar').replace('caixeiros viajantes', 'caixeiros_viajantes').replace('canto do sabiá', 'canto_do_sabiá').replace('canto provinciano', 'canto_provinciano').replace('canção pra ninar curumim', 'canção_pra_ninar_curumim').replace('capitães da areia', 'capitães_da_areia').replace('carlinhos brown', 'carlinhos_brown').replace('carnaval de 2008', 'carnaval_de_2008').replace('castelo do plácido', 'castelo_do_plácido').replace('castelo encantado', 'castelo_encantado').replace('centro dragão do mar', 'centro_dragão_do_mar').replace('cidade 2000', 'cidade_2000').replace('cidade dos funcionários', 'cidade_dos_funcionários').replace('cidadão instigado', 'cidadão_instigado').replace('circo escola', 'circo_escola').replace('classe média', 'classe_média').replace('classe média alta', 'classe_média_alta').replace('clube da esquina no 2', 'clube_da_esquina_no_2').replace('cláudio pereira', 'cláudio_pereira').replace('companhia bate palmas', 'companhia_bate_palmas').replace('concha acústica', 'concha_acústica').replace('confederação do equador', 'confederação_do_equador').replace('conjunto palmeiras', 'conjunto_palmeiras').replace('crescer com arte', 'crescer_com_arte').replace('cultura do povo', 'cultura_do_povo').replace('cultura indígena', 'cultura_indígena').replace('cultura insubmissa', 'cultura_insubmissa').replace('dado villa-lobos', 'dado_villa_lobos').replace('dança de rua', 'dança_de_rua').replace('davi duarte', 'davi_duarte').replace('dedé evangelista', 'dedé_evangelista').replace('deusas de iracema', 'deusas_de_iracema').replace('ditador do estado novo', 'ditador_do_estado_novo').replace('ditadura militar', 'ditadura_militar').replace('dona maria', 'dona_maria').replace('dona mocinha', 'dona_mocinha').replace('doutor barros pinho', 'doutor_barros_pinho').replace('dragão do mar', 'dragão_do_mar').replace('década de 1940', 'década_de_1940').replace('década de setenta', 'década_de_setenta').replace('edifício são pedro', 'edifício_são_pedro').replace('elvis presley silva', 'elvis_presley_silva').replace('esquina do brasil', 'esquina_do_brasil').replace('estados unidos', 'estados_unidos').replace('estrela do norte', 'estrela_do_norte').replace('ex-prefeito roberto cláudio', 'ex_prefeito_roberto_cláudio').replace('fausto nilo', 'fausto_nilo').replace('favela da zé bastos', 'favela_da_zé_bastos').replace('feira do benfica', 'feira_do_benfica').replace('femme bateau', 'femme_bateau').replace('festa católica', 'festa_católica').replace('festa da quermesse', 'festa_da_quermesse').replace('finais de semana', 'finais_de_semana').replace('flávio torres', 'flávio_torres').replace('futuro e memória', 'futuro_e_memória').replace('getúlio vargas', 'getúlio_vargas').replace('governo do estado', 'governo_do_estado').replace('governo luizianne', 'governo_luizianne').replace('grande circular', 'grande_circular').replace('grupo siriará de literatura', 'grupo_siriará_de_literatura').replace('gruta do seu zairton', 'gruta_do_seu_zairton').replace('guardas municipais', 'guardas_municipais').replace('holocausto capital', 'holocausto_capital').replace('igreja católica', 'igreja_católica').replace('igreja de fátima', 'igreja_de_fátima').replace('igreja do mucuripe', 'igreja_do_mucuripe').replace('iv festival da música popular cearense', 'iv_festival_da_música_popular_cearense').replace('jangada são pedro', 'jangada_são_pedro').replace('john lennon', 'john_lennon').replace('jornal o pão', 'jornal_o_pão').replace('josé capello', 'josé_capello').replace('josé de alencar', 'josé_de_alencar').replace('lugar de partida', 'lugar_de_partida').replace('luiz assumpção', 'luiz_assumpção').replace('luiz carlos porto', 'luiz_carlos_porto').replace('lúcio ricardo', 'lúcio_ricardo').replace('major facundo', 'major_facundo').replace('maluco beleza', 'maluco_beleza').replace('manuel olímpio meire', 'manuel_olímpio_meire').replace('manuel pereira da silva (manuel preto)', 'manuel_pereira_da_silva_(manuel_preto)').replace('marca palma', 'marca_palma').replace('marcos dias', 'marcos_dias').replace('marcos francisco', 'marcos_francisco').replace('marcus (dias)', 'marcus_(dias)').replace('marcus dias', 'marcus_dias').replace('marina park', 'marina_park').replace('mercado central', 'mercado_central').replace('mercado público da aerolândia', 'mercado_público_da_aerolândia').replace('mestre jerônimo', 'mestre_jerônimo').replace('mestre jerônimo andré de sousa', 'mestre_jerônimo_andré_de_sousa').replace('meu corpo minha embalagem todo gasto na viagem', 'meu_corpo_minha_embalagem_todo_gasto_na_viagem').replace('milton nascimento', 'milton_nascimento').replace('morro das palmeiras', 'morro_das_palmeiras').replace('morro marajaitiba', 'morro_marajaitiba').replace('movimentos sociais', 'movimentos_sociais').replace('mucambo cafundó', 'mucambo_cafundó').replace('mulher barco', 'mulher_barco').replace('márcio catunda', 'márcio_catunda').replace('mário gomes', 'mário_gomes').replace('nada sou', 'nada_sou').replace('new york', 'new_york').replace('nomes de ruas', 'nomes_de_ruas').replace('orson welles', 'orson_welles').replace('oswald de andradequarenta e cinco, do segundo tempo', 'oswald_de_andradequarenta_e_cinco,_do_segundo_tempo').replace('padaria espiritual', 'padaria_espiritual').replace('palma linha', 'palma_linha').replace('palmas fashion', 'palmas_fashion').replace('passarela na br 116', 'passarela_na_br_116').replace('passeio público', 'passeio_público').replace('pedro breculê', 'pedro_breculê').replace('pedro pereira', 'pedro_pereira').replace('pescadores cearenses', 'pescadores_cearenses').replace('pessoas de favela', 'pessoas_de_favela').replace('petrúcio maia', 'petrúcio_maia').replace('pinturas de rosto', 'pinturas_de_rosto').replace('polícia militar', 'polícia_militar').replace('ponte dos ingleses', 'ponte_dos_ingleses').replace('ponte metálica', 'ponte_metálica').replace('porto do mucuripe', 'porto_do_mucuripe').replace('pouso do sabiá', 'pouso_do_sabiá').replace('povo da universidade', 'povo_da_universidade').replace('povo do benfica', 'povo_do_benfica').replace('povo do carnaval', 'povo_do_carnaval').replace('praia de iracema', 'praia_de_iracema').replace('praia do futuro', 'praia_do_futuro').replace('praia do lido', 'praia_do_lido').replace('praia do peixe', 'praia_do_peixe').replace('praia dos crush', 'praia_dos_crush').replace('praça da gentilândia', 'praça_da_gentilândia').replace('praça do carmo', 'praça_do_carmo').replace('prefeitura de fortaleza', 'prefeitura_de_fortaleza').replace('programa de erradicação do trabalho infantil', 'programa_de_erradicação_do_trabalho_infantil').replace('projeto um canto em cada canto', 'projeto_um_canto_em_cada_canto').replace('proteção ambiental', 'proteção_ambiental').replace('pré carnaval', 'pré_carnaval').replace('pré-carnaval', 'pré_carnaval').replace('pôr do sol', 'pôr_do_sol').replace('quermesse no mucuripe', 'quermesse_no_mucuripe').replace('questão ambiental', 'questão_ambiental').replace('quinto whisky', 'quinto_whisky').replace('quinze anos', 'quinze_anos').replace('raimundo correia lima (tatá)', 'raimundo_correia_lima_.replace(tatá)').replace('recém-inaugurado', 'recém_inaugurado').replace('renato russo', 'renato_russo').replace('riacho pajeú', 'riacho_pajeú').replace('ricardo bezerra', 'ricardo_bezerra').replace('rio de janeiro', 'rio_de_janeiro').replace('rock projetivo', 'rock_projetivo').replace("rock'n roll", 'rock_n_roll').replace('roda de samba', 'roda_de_samba').replace('rodger rogério', 'rodger_rogério').replace('roqueiro do montese', 'roqueiro_do_montese').replace('rua artur timóteo', 'rua_artur_timóteo').replace('rua joão cordeiro', 'rua_joão_cordeiro').replace('rádio universitária', 'rádio_universitária').replace('sabiá sabia', 'sabiá_sabia').replace('sabiá sabiaguaba', 'sabiá_sabiaguaba').replace('salão de idosos', 'salão_de_idosos').replace('samba canção', 'samba_canção').replace('samba-canção', 'samba_canção').replace('segunda-feira', 'segunda_feira').replace('seu zairton', 'seu_zairton').replace('sinal de trânsito', 'sinal_de_trânsito').replace('sobre a ponte e femme bateau', 'sobre_a_ponte_e_femme_bateau').replace('sul maravilha', 'sul_maravilha').replace('são sebastião', 'são_sebastião').replace('sérgio pinheiro', 'sérgio_pinheiro').replace('sérvulo esmeraldo', 'sérvulo_esmeraldo').replace('súplica cearense', 'súplica_cearense').replace('teatro universitário', 'teatro_universitário').replace('terminal da parangaba', 'terminal_da_parangaba').replace('treze de maio', 'treze_de_maio').replace('trinta anos', 'trinta_anos').replace('tráfico de garotas', 'tráfico_de_garotas').replace('tupi guarani', 'tupi_guarani').replace('turismo sexual', 'turismo_sexual').replace('tv educativa', 'tv_educativa').replace('um sol para cada um', 'um_sol_para_cada_um').replace('vii festival internacional da canção', 'vii_festival_internacional_da_canção').replace('visão social', 'visão_social').replace('válvula de escape', 'válvula_de_escape').replace('zé bezerra', 'zé_bezerra').replace('zé walter', 'zé_walter').replace('ângela linhares', 'ângela_linhares').replace('época de ouro', 'época_de_ouro').replace('órbita bar', 'órbita_bar')

In [ ]:
# deixar letras de músicas em caixa baixa, criar a coluna '['letras_musicas'] e aplicar função processingTxt

df_inspiracao['inspiracao_musica'] = df_inspiracao['inspiração_copia'].str.lower()
df_inspiracao['inspiracao_musica'] = df_inspiracao['inspiracao_musica'].fillna('-').apply(lambda x: processingTxt(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# aplicar função encontrarPalavras na coluna df_inspiracao['Inspiração']

nuvem_inspiracao = encontrarPalavras(df_inspiracao['inspiracao_musica'])

In [ ]:
# agrupar palavras por repetição

df_nuvem_inspiracao = nuvem_inspiracao.groupby(by='palavras').sum().sort_values(by='repetições', ascending=False).reset_index()

In [ ]:
# excluir célula com espaço em branco
df_nuvem_inspiracao = df_nuvem_inspiracao[df_nuvem_inspiracao['palavras'] != '']

In [ ]:
# visualizar palavras mais repetidas

df_nuvem_inspiracao.head()

,palavras,repetições
0,música,91
1,a gente,61
2,fortaleza,39
3,cidade,39
5,canção,28


In [ ]:
# total de palavras
df_nuvem_inspiracao['palavras'].nunique()

2007

In [ ]:
# total de palavras que se repetem
df_nuvem_inspiracao[df_nuvem_inspiracao['repetições'] != 1]['palavras'].nunique()

735

In [ ]:
# total de palavras com ocorrência única
df_nuvem_inspiracao[df_nuvem_inspiracao['repetições'] == 1]['palavras'].nunique()

1272

In [ ]:
# salvar arquivo em .csv

df_nuvem_inspiracao.to_csv('df_nuvem_inspiracao.csv', sep=',', encoding='utf-8', index=False)

## Encontrando os bairros de Fortaleza mais cantados  <a name="bairros"></a>

### Quantidade de músicas por bairros <a name="quantidade"></a>

In [ ]:
# criar dataframe com filtro de colunas para visualizar os bairros cantados em cada período

musica_ano = arquivo[['Ano', 'Autoria', 'Nome da música', 'musica', 'Bairro', 'Latitude', 'Longitude', 'idYT']]

In [ ]:
# transformar 'Ano' em int
musica_ano['Ano'] = musica_ano['Ano'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
bairros_fortaleza = pd.read_csv('arquivo_bairros_fortaleza.csv') 
# Fonte do arquivo: mapas.fortaleza.ce.gov.br
bairros_fortaleza['the_geom'] = gpd.GeoSeries.from_wkt(bairros_fortaleza['the_geom'])
gBairros = gpd.GeoDataFrame(bairros_fortaleza, geometry='the_geom') 

In [ ]:
def getBairro(Point, coluna_desejada_gBairros):
    query = gBairros[gBairros['the_geom'].contains(Point)]
    if query.shape[0] > 0:
        return query.iloc[0][coluna_desejada_gBairros]
    else:
        return 'Não identificado'

In [ ]:
musica_ano['Latitude'] = musica_ano['Latitude'].astype('float')
musica_ano['Longitude'] = musica_ano['Longitude'].apply(lambda x: str(x).split(',')[0]).astype('float')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
musica_ano['Point'] = musica_ano.apply(lambda x: Point(x['Longitude'],x['Latitude']), axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
musica_ano['bairro_name'] = musica_ano.apply(lambda x: getBairro(x['Point'],'bairro'), axis=1)
musica_ano['bairro_GID'] = musica_ano.apply(lambda x: getBairro(x['Point'],'GID'), axis=1) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
musica_ano['comparacao'] = np.where(musica_ano['Bairro'] == musica_ano['bairro_name'], True, False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
musica_ano.to_csv('musica_bairros_calculados.csv', index=False)

In [ ]:
# musica_ano[musica_ano['comparacao'] == False]

In [ ]:
bairros_identificados = musica_ano[musica_ano['bairro_name'] != 'Não identificado']
# bairros_identificados.head()

In [ ]:
# encontrando o total de músicas
total_musicas = musica_ano['bairro_name'].value_counts().rename_axis('Bairros').to_frame('Total de músicas').reset_index()['Total de músicas'].sum()

In [ ]:
# encontrando o número de músicas cujos bairros foram identificados
musicas_bairros_identificados = bairros_identificados['bairro_name'].value_counts().rename_axis('Bairros').to_frame('Total de músicas').reset_index()['Total de músicas'].sum()

In [ ]:
print('Entre o total de', total_musicas,'músicas presentes na base de dados do Fortaleza em música,', musicas_bairros_identificados,'têm identificação dos bairros.')

Entre o total de 79 músicas presentes na base de dados do Fortaleza em música, 73 têm identificação dos bairros.


In [ ]:
df_bairros_identificados = bairros_identificados[['bairro_name', 'musica']].groupby(by=['bairro_name']).count().sort_values(by='musica', ascending=False).reset_index()

In [ ]:
# encontrando os três bairros mais cantados
# df_bairros_identificados[0:3]

In [ ]:
# Desde quando esses três bairros mais cantados aparecem nas canções?

# bairros_identificados[['Ano', 'musica', 'bairro_name']][(bairros_identificados['bairro_name'] == 'Praia de Iracema') | (bairros_identificados['bairro_name'] == 'Mucuripe') | (bairros_identificados['bairro_name'] == 'Centro')].sort_values(by='Ano').head()

In [ ]:
# total de músicas que corespondem aos bairros mais cantados (Centro, Praia de Iracema e Mucuripe)
bairros_mais_cantados = df_bairros_identificados[0:3]['musica'].sum()
bairros_mais_cantados

29

In [ ]:
# total de músicas que corespondem aos outros bairros, exceto os mais cantados, para checar o total
demais_bairros = df_bairros_identificados[3:26]['musica'].sum()
demais_bairros

44

In [ ]:
# encontrando o percentual das músicas que se referem aos principais bairros, em relação ao total de músicas com bairros identificados
print('As músicas que cantam Centro, Praia de Iracema e Mucuripe correspondem aproximadamente a', round(bairros_mais_cantados/musicas_bairros_identificados*100,0), '% do total das músicas com bairros identificados')

As músicas que cantam Centro, Praia de Iracema e Mucuripe correspondem aproximadamente a 40.0 % do total das músicas com bairros identificados


In [ ]:
# salvar arquivo .csv para visualização
df_bairros_identificados.to_csv('df_musicas_por_bairros.csv', sep=',', encoding='utf-8', index=False)

### E agora também por período  <a name="período"></a>

In [ ]:
bairros_identificados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 78
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Ano             73 non-null     int64  
 1   Autoria         73 non-null     object 
 2   Nome da música  73 non-null     object 
 3   musica          73 non-null     object 
 4   Bairro          73 non-null     object 
 5   Latitude        73 non-null     float64
 6   Longitude       73 non-null     float64
 7   idYT            73 non-null     object 
 8   Point           73 non-null     object 
 9   bairro_name     73 non-null     object 
 10  bairro_GID      73 non-null     object 
 11  comparacao      73 non-null     bool   
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 6.9+ KB


In [ ]:
# criar coluna de década

musicas_1960 = bairros_identificados[bairros_identificados['Ano'] == 1969]
musicas_1960['Década'] = '1960'

musicas_1970 = bairros_identificados[bairros_identificados['Ano'].isin([1970, 1972, 1974, 1976, 1978, 1979])]
musicas_1970['Década'] = '1970'

musicas_1980 = bairros_identificados[bairros_identificados['Ano'].isin([1980, 1988])]
musicas_1980['Década'] = '1980'

musicas_1990 = bairros_identificados[bairros_identificados['Ano'].isin([1990, 1993, 1994, 1996])]
musicas_1990['Década'] = '1990'

musicas_2000 = bairros_identificados[bairros_identificados['Ano'].isin([2000, 2002, 2003, 2004, 2005, 2007, 2008, 2009])]
musicas_2000['Década'] = '2000'

musicas_2010 = bairros_identificados[bairros_identificados['Ano'].isin([2010, 2011, 2012, 2013, 2019, 2015, 2016, 2017, 2018])]
musicas_2010['Década'] = '2010'

musicas_2020 = bairros_identificados[bairros_identificados['Ano'].isin([2020, 2021])]
musicas_2020['Década'] = '2020'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [ ]:
# concatenar dataframes anteriores
df_musicas_decadas = pd.concat([musicas_1960, musicas_1970, musicas_1980, musicas_1990, musicas_2000, musicas_2010, musicas_2020])

In [ ]:
# df_musicas_decadas[['Década', 'bairro_name', 'musica']].groupby(by='Década').nunique().reset_index()

In [ ]:
# agora vamos visualizar os dados sem os bairros mais cantados

df_musicas_decadas[['Ano', 'musica', 'bairro_name']][~df_musicas_decadas['bairro_name'].isin(['Praia de Iracema', 'Mucuripe', 'Centro'])].head()

,Ano,musica,bairro_name
7,1972,"Beira Mar, por Ednardo",Meireles
14,1979,"Brincando É Que Se Aprende, por Dominguinhos e...",Aldeota
43,1978,"Maraponga, por Ricardo Bezerra",Parangaba
65,1972,"Terral, por Ednardo",Praia do Futuro I
23,1996,"Cinema Noir, por Mona Gadelha",Praia do Futuro I


## Coleta YouTube DATA API

In [ ]:
api = 'api_key'
youtube = build('youtube','v3', developerKey=api)

In [ ]:
def getStatistics(video_id):
  res = youtube.videos().list(part=['snippet','statistics'], id=video_id).execute()
  return res['items'][0]

In [ ]:
arquivo['youtubeData'] = arquivo.idYT.apply(lambda x: getStatistics(x))

In [ ]:
arquivo['snippetYT'] = arquivo['youtubeData'].apply(lambda x: x['snippet'])
arquivo['statisticsYT'] = arquivo['youtubeData'].apply(lambda x: x['statistics'])

In [ ]:
for key in list(arquivo['statisticsYT'][0].keys()):
  arquivo[key] = arquivo['statisticsYT'].apply(lambda x: x[key]).astype('int')

In [ ]:
f'As 79 músicas do App já acumulam {arquivo.viewCount.sum()} visualizações, {arquivo.likeCount.sum()} likes e {arquivo.commentCount.sum()} comentários no Youtube'

'As 79 músicas do App já acumulam 2715296 visualizações, 37027 likes e 1768 comentários no Youtube'

In [ ]:
top10 = arquivo.sort_values(by=['viewCount'],ascending = False).filter(['Nome da música','viewCount','likeCount','commentCount','link','Ano','Autoria']).head(10)
# top10

In [ ]:
# salvar arquivo em .csv
top10.to_csv('10mais.csv', index = False)

In [ ]:
# filtrar arquivo para criação do mapa no Observable
dataset_observable = musica_ano[['Nome da música', 'Autoria', 'Ano', 'Latitude', 'Longitude', 'bairro_name', 'idYT']]

In [ ]:
# salvar arquivo em .csv
dataset_observable.to_csv('dataset_observable.csv', index=False)